In [1]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [2]:
!pip install -q toolz
!pip install -q wandb

In [20]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [5]:
csv = pd.read_csv("ZINC_310k.csv")

In [6]:
smiles = csv['SMILES']

In [7]:
smiles = smiles[:60000]

In [8]:
labels = torch.tensor(csv['LogP'][:60000]).float()

In [9]:
# labels = torch.tensor(csv['LogP']).float()

In [10]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [13]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

100%|██████████| 12/12 [21:37<00:00, 108.16s/it]


In [12]:
# with open('runner.xml', 'rb') as f:
#     runner = pickle.load(f)

In [10]:
with open('runner_20.xml', 'rb') as f:
    runner = pickle.load(f)

In [11]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [12]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [13]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [14]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])

X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [15]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [16]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [17]:
with open('train.npy', 'wb') as f:
    np.save(f, X_train)
    
with open('test.npy', 'wb') as f:
    np.save(f, X_test)
    
with open('validation.npy', 'wb') as f:
    np.save(f, X_Val)

In [18]:
# with open('train.npy', 'rb') as f:
#     X_train = np.load(f, allow_pickle=True)

# with open('test.npy', 'rb') as f:
#     X_test = np.load(f, allow_pickle=True)

# with open('validation.npy', 'rb') as f:
#     X_Val = np.load(f, allow_pickle=True)

In [ ]:
print("Training model...")
runner.train_gen_pred(
    X_train,
    L_train,
    X_test,
    L_test,
    X_Val,
    L_Val,
    load_epoch= 0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
    batch_size=conf["batch_size"],
    num_workers=conf["num_workers"],
    label_pct=0.5
)

Training model...
Model #Params: 4732K
[Train][100] Alpha: 265.000, Beta: 0.126, Loss: 322.77, KL: 23.94, MAE: 0.78348, Word Loss: 97.17, Topo Loss: 27.58, Assm Loss: 9.39, Pred Loss: 0.70, Word: 25.67, Topo: 78.03, Assm: 52.75, PNorm: 101.06, GNorm: 50.00
[Train][200] Alpha: 265.000, Beta: 0.126, Loss: 186.78, KL: 23.87, MAE: 0.56106, Word Loss: 64.40, Topo Loss: 16.86, Assm Loss: 8.65, Pred Loss: 0.35, Word: 42.44, Topo: 87.43, Assm: 56.29, PNorm: 104.97, GNorm: 50.00
[Train][300] Alpha: 265.000, Beta: 0.126, Loss: 184.16, KL: 16.84, MAE: 0.58014, Word Loss: 56.10, Topo Loss: 16.29, Assm Loss: 8.67, Pred Loss: 0.38, Word: 51.03, Topo: 87.72, Assm: 56.93, PNorm: 108.07, GNorm: 50.00
[Train][400] Alpha: 265.000, Beta: 0.126, Loss: 134.62, KL: 18.29, MAE: 0.43410, Word Loss: 53.64, Topo Loss: 12.31, Assm Loss: 8.30, Pred Loss: 0.22, Word: 52.74, Topo: 91.03, Assm: 59.49, PNorm: 111.15, GNorm: 50.00
[Train][500] Alpha: 265.000, Beta: 0.126, Loss: 119.79, KL: 15.77, MAE: 0.37938, Word Los

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)